In [ ]:
# Disser er hentet fra eksempelet vi fikk utdelt. Kommenterer ut noen da jeg ikke vet hva de brukes til.
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from fitparse import FitFile
from os import listdir
from os.path import isfile, join, splitext
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as md

Fetch all the files in a simple for-loop

In [ ]:
location_string = '../Fitfiles/'
participant_files = []
for f in listdir(location_string):
    if isfile(location_string+f):
# participant_files.append(location_string+'/'+f) #hele location
        participant_files.append(f) #kun selve filen
# print(participant_files)

Find all message names

In [ ]:
fitmessages = []

for participant_file in participant_files:
    fitfile = FitFile(location_string+participant_file)

    for rec in fitfile:
        fitmessages.append(rec.name)
print(set(fitmessages))


Crate a txt file with all messagenames

In [ ]:
f = open('messagenames50338.txt', 'w')
fitfile = FitFile(location_string+'50338.FIT')
for rec in fitfile:
    f.write(rec.name)
    f.write('\n')
f.close()

#regex \bhrv\n\bhrv

Show all content of all hrv messages

In [ ]:
fitmessages = []

for participant_file in participant_files:
    fitfile = FitFile(location_string+participant_file)

    for rec in fitfile:
        if rec.name=='hrv':
            fitmessages.append(rec.get_values)
print(set(fitmessages))

Simple test of first found hrv message

In [ ]:
fitfile = FitFile(location_string+'50048.FIT')

for rec in fitfile:
    if rec.name=='hrv':
        print(rec.get_values)
        print(rec.get_value('time'))
        print(rec.get_value('time')[1])
        test = rec.get_value('time')
        for i in range(len(test)):
            if test[i] == None:
                print('yo!')
            else:
                print('no yo!')
        break

Defining methods to read messages from fitfile

In [ ]:
# hrv test
def speed_corr(speed):
    return speed if isinstance(speed, float) else 0

def power_corr(power):
    return power if isinstance(power, int) else 0

def read_record(rec):
    global curr_timestamp
    global curr_stop
    global curr_active
    if curr_stop == 1 and speed_corr(rec.get_value('speed')) > 7:
        curr_stop = 0
    if ((speed_corr(rec.get_value('speed')) > 0) & (pd.isnull(rec.get_value('power'))) or ((speed_corr(rec.get_value('speed')) > 0) & (power_corr(rec.get_value('power'))> 0))):
        curr_active = 1
    curr_timestamp = rec.get_value('timestamp')
    return (curr_timestamp,
            1, #type = 1
            curr_lap,
            curr_stop, 
            curr_active,
            rec.get_value('speed'), 
            rec.get_value('enhanced_speed'),
            rec.get_value('distance'), 
            np.NaN, #HRV
            rec.get_value('heart_rate'), 
            rec.get_value('accumulated_power'), 
            rec.get_value('power'), 
            rec.get_value('position_lat'), 
            rec.get_value('position_long'), 
            rec.get_value('altitude'), 
            rec.get_value('enhanced_altitude'), 
            rec.get_value('cadence'), 
            rec.get_value('factional_cadence'), 
            rec.get_value('left_right_balnce'), 
            rec.get_value('left_torque_effectiveness'), 
            rec.get_value('right_torque_effectiveness'), 
            rec.get_value('left_pedal_smoothness'), 
            rec.get_value('right_pedal_smoothness'), 
            rec.get_value('temperature'))

def read_lap(rec):
    global curr_timestamp
    global curr_lap
    global curr_stop
    curr_lap += 1
    curr_stop = 1
    curr_timestamp = rec.get_value('timestamp')
    return (curr_timestamp, 
            2, # type = 2
            curr_lap, 
            curr_stop, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            rec.get_value('total_distance'),
            np.NaN, #HRV 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN)

def read_hrv_timestepper(rec):
    #global timestepper
    # NOT DONE
    return (curr_timestamp, 
            3, # type = 2
            curr_lap, 
            curr_stop, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            rec.get_value('total_distance'), 
            np.NaN, #HRV
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN)

def read_hrv_mean(rec):
    numerator = 0
    denominator = 0
    for a in range(5):
        if rec.get_value('time')[a] != None:
            denominator += 1
            numerator += rec.get_value('time')[a]
    if denominator != 0:
        mean_hrv = numerator/denominator
    else:
        mean_hrv = 0
    return (curr_timestamp,
            3, # type = 3
            curr_lap, 
            curr_stop, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            rec.get_value('total_distance'), 
            mean_hrv, #HRV
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN)

Rewrote given generator for ease of reading and expanding

In [ ]:
#skriver om fitgen for å enklere kunne lese og legge til nye messages
def fitgenerator(fitfile):
    for rec in fitfile.get_messages(['record', 'lap', 'hrv']):
        if rec.name=='record':
            yield read_record(rec)
        elif rec.name=='lap':
            yield read_lap(rec)
        else:
            yield read_hrv_mean(rec)

Initializing key variables

In [ ]:
fitgen_cols = ['timestamp', 'type', 'lap', 'stop', 'active', 'speed', 'enhanced_speed', 'distance', 'hrv',                           'heart_rate', 
               'accumulated_power', 'power', 'position_lat', 'position_long', 
               'altitude', 'enhanced_altitude', 'cadence', 'factional_cadence', 
               'left_right_balnce', 'left_torque_effectiveness', 'right_torque_effectiveness', 
               'left_pedal_smoothness', 'right_pedal_smoothness', 'temperature']
all_dfs = []
all_stops = []
all_hrv = []
all_dStops = []
all_rDfs = []

Convert all messages (defined by generator) to dataframe

In [ ]:
for participant_file in participant_files:
    fitfile = FitFile(location_string+participant_file)
    print(participant_file, 'start')
    
    curr_lap = 0
    curr_stop = 0
    curr_active = 0
    #curr_timestamp - tror ikkje me trenge denne

    fitgen = fitgenerator(fitfile)

    allraw = list(rec for rec in fitfile.get_messages(['record', 'lap', 'hrv']))

    print(participant_file, 'read')
    
    part_df = pd.DataFrame(fitgen, columns=fitgen_cols)
    all_dfs.append(part_df[part_df['type']==1].set_index('timestamp'))
    all_stops.append(part_df[part_df['type']==2].set_index('timestamp'))
    all_hrv.append(part_df[part_df['type']==3].set_index('timestamp'))

    print(participant_file, 'converted')

Create txt file with all hrv messages (incorrectly due to bad hrv reading method)

In [ ]:
f = open('all_hrv.txt', 'w')
for element in all_hrv:
    f.write(element.to_string())
    f.write('\n')
f.close()

In [ ]:
fitmessages = []

for participant_file in participant_files:
    fitfile = FitFile(location_string+participant_file)
    print('\n')
    for rec in fitfile:
        if rec.name=='file_id':
            print(rec.get_values())


In [ ]:
fitmessages = []

for participant_file in participant_files:
    fitfile = FitFile(location_string+participant_file)
    for rec in fitfile:
        if rec.name=='hr':
            fitmessages.append(participant_file)
print(set(fitmessages))

In [ ]:
#{'50880.FIT', '50499.FIT'}, bruker HR messages.
